In [3]:
import nltk,os
import requests
import re
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

path_df = "Pickles/News_dataset.pickle"

with open(path_df, 'rb') as data:
    ds = pickle.load(data)

if type(14) == int:
    print("s")

s


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

path_df = "Pickles/News_dataset.pickle"

with open(path_df, 'rb') as data:
    ds = pickle.load(data)

ids  = [i for i in range(len(ds))]
idf = pd.DataFrame(ids)
ds = pd.concat([ds, ids],axis=1, ignore_index=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['Content'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
    
print('done!')

def item(id):
    return ds.loc[ds['id'] == id]['Title'].tolist()[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

recommend(item_id=11, num=10)

In [ ]:
import pickle, time, os
from datetime import datetime

path_df = "Pickles/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

doc_all = {}

for i in range(len(df)):
    doc_all[df.loc[i]['Title']] = df.loc[i]['Content'].split()
    
print(len(doc_all))

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
start = time.time()
#create bag words
bag_words =[] # declare bag_words is a list
for doc in doc_all.keys():
    bag_words += doc_all[doc]
bag_words=set(bag_words)
#calculate idf for every word in bag_words
bag_words_idf={}  
bag_words_len = len(bag_words)
bag_word_10 = round(bag_words_len/10,0)
print("the number of term in bag_word", bag_words_len)

In [ ]:
import nltk,os
import requests
import re, pickle
import pandas as pd
nltk.download('punkt')
import numpy as np

path_df = "./Pickles/tfidf_for_recommend.pickle"

with open(path_df, 'rb') as data:
    tfidf = pickle.load(data)

path_df = "./Pickles/bag_words_idf.pickle"

with open(path_df, 'rb') as data:
    bag_words_idf = pickle.load(data)

In [ ]:
query="bad trump"
query_token_raw= nltk.word_tokenize(query)
query_token = [term for term in query_token_raw if term in bag_words]

tfidf_query =compute_tfidf_query(query_token,bag_words_idf) #calculate tfidf for query text
print(tfidf_query)

In [ ]:
# add tfidf of query text to tfidf of all doc and convert to dataframe
tfidf["query"] = tfidf_query

tfidf_df = pd.DataFrame(tfidf).transpose()
tfidf_df= tfidf_df.fillna(0) # replace all NaN by zero

from scipy.spatial.distance import cosine
cosine_sim ={}
for row in tfidf_df.index:
    if row != "query":
        cosine_sim[row]= 1-cosine(tfidf_df.loc[row],tfidf_df.loc["query"])

# the top 10 relevant document
cosine_sim_top10 = dict(sorted(cosine_sim.items(), key=lambda item: item[1],reverse=True)[:10])
print(cosine_sim_top10)

current_time = now.strftime("%H:%M:%S")
print("Finish tfidf processing at", current_time)
spent = time.time() - start
print("\nTotal spent time: "+str(spent) +"sec\n")

In [ ]:
#plot barchart
import matplotlib.pyplot as plt
data = cosine_sim_top10
plt.barh(range(len(data)), list(data.values()), align='center', alpha=0.8)
plt.yticks(range(len(data)), list(data.keys())) # label for y axis
plt.xlabel('Smimilarity score')
plt.ylabel('news')

# save graph

plt.savefig("./recommand_barchart.png", bbox_inches='tight', dpi=600)
plt.show()